In [37]:
import numpy as np
import xarray as xr
import netCDF4 as nc
import matplotlib.pyplot as plt
import os
import socket

In [28]:
def compute_global_mean(ds):
    wts = ds.profile_weight/ds.profile_weight.sum()
    return(ds.weighted(wts).mean(["site"]))

In [15]:
#copy directory here
os.chdir('/Users/emmalayton/Desktop/senior_thesis/RFMIP-IRF_intake_catalogue')

In [16]:
from intake import open_catalog
cat = open_catalog(os.getcwd() + '/main.yml')

In [17]:
xr.concat([compute_global_mean(cat.parameterized.GISS_E3.p1f1[f].to_dask()) for f in ["rlu", "rld", "rsu", "rsd"]], 
          dim="field", data_vars = "different")

<xarray.Dataset>
Dimensions:         (expt: 18, level: 61)
Dimensions without coordinates: expt, level
Data variables:
    rld             (expt, level) float32 0.001991 0.2302 0.4592 ... 314.3 314.9
    rsu             (expt, level) float32 46.46 46.46 46.47 ... 30.34 30.3 30.28
    rsd             (expt, level) float32 335.2 335.1 335.0 ... 247.1 246.9
    plev            (level) float32 0.01 20.0 38.43 ... 9.839e+04 9.862e+04
    rlu             (expt, level) float32 262.4 262.5 262.6 ... 392.1 391.6
    profile_weight  float64 0.01053

In [33]:
benchmark = xr.concat([compute_global_mean(cat.benchmark.LBLRTM.p1f1[f].to_dask()) 
                               for f in ["rlu", "rld", "rsu", "rsd"]], 
                               dim="field", data_vars = "different")

In [44]:
gbms = {f'{m}_{r}': xr.concat([compute_global_mean(cat.parameterized[m][r][f].to_dask()) 
                               for f in ["rlu","rld","rsu","rsd"], 
                               dim="field", data_vars = "different") 
        for m in list(cat.parameterized) for r in list(cat.parameterized[m])}
#does it make sense to split rlu, rld, rsu, rsd so that it doesnt time out?

ConnectionError: HTTPSConnectionPool(host='esgf-data1.llnl.gov', port=443): Max retries exceeded with url: /thredds/dodsC/css03_data/CMIP6/RFMIP/NASA-GISS/GISS-E2-1-G/rad-irf/r1i1p1f1/Efx/rld/gn/v20191230/rld_Efx_GISS-E2-1-G_rad-irf_r1i1p1f1_gn.nc.dods?profile_weight%5B0:1:99%5D (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x14a977190>: Failed to establish a new connection: [Errno 60] Operation timed out'))

In [ ]:
gbme = {k: (gbms[k] - benchmark).isel(level = [0, 60]) for k in gbms.keys()}

In [90]:
gbmf = {k: (gbme['expt'][1:18]-gbme['expt'][0]) for k in gbms.keys()}